# ECB cut-and-paste
Write a k=v parsing routine, as if for a structured cookie. The routine should take:

```
foo=bar&baz=qux&zap=zazzle
```
... and produce:
```
{
  foo: 'bar',
  baz: 'qux',
  zap: 'zazzle'
}
```
(you know, the object; I don't care if you convert it to JSON).

Now write a function that encodes a user profile in that format, given an email address. You should have something like:
```
profile_for("foo@bar.com")
```
... and it should produce:
```
{
  email: 'foo@bar.com',
  uid: 10,
  role: 'user'
}
```
... encoded as:
```
email=foo@bar.com&uid=10&role=user
```
Your "profile_for" function should not allow encoding metacharacters (& and =). Eat them, quote them, whatever you want to do, but don't let people set their email address to "foo@bar.com&role=admin".

Now, two more easy functions. Generate a random AES key, then:

1. Encrypt the encoded user profile under the key; "provide" that to the "attacker".
1. Decrypt the encoded user profile and parse it.

Using only the user input to profile_for() (as an oracle to generate "valid" ciphertexts) and the ciphertexts themselves, make a role=admin profile.

In [2]:
from importlib import reload
import cryptopals
reload(cryptopals)
from cryptopals import *

In [48]:
def kv_parser(s):
    return {i[:i.find('=')] : i[i.find('=')+1:] for i in s.strip().split('&') if i.find('=') > 0}

import uuid
def profile_for(useremail):
    return f'email={useremail.replace("&", "").replace("=", "")}&uid={uuid.uuid4()}&role=user'

def unpad_pkcs7(b, block_size=16):
    '''Unpad a bytearray or string using the PKCS #7 scheme.'''
    last_byte = ord(to_bytes(b[-1:]))
    return b[:-last_byte]

def encrypt_user_profile(useremail, key):
    return encrypt_AES_128_ECB(profile_for(useremail), key)

def decrypt_user_profile(ciphertext, key):
    return kv_parser(unpad_pkcs7(decrypt_AES_128_ECB(ciphertext, key)))

def construct_admin_ciphertext():
    def oracle(useremail):
        return encrypt_user_profile(useremail, consistent_unknown_key)

    # probe for last block containing 'admin' and padding
    probe_email = (AES.block_size - len('email=')) * chr(0) + pad_pkcs7('admin').decode()
    admin_last_block = oracle(probe_email)[AES.block_size : 2*AES.block_size]

    # probe block length
    malicious_email = 'm@licio.us'
    base_len = len(oracle(malicious_email))
    while True:
        malicious_email = 'x'+malicious_email
        if len(oracle(malicious_email)) > base_len:
            break
    malicious_email = 'x'*len('user') + malicious_email

    # copy-and-paste admin block
    ct = oracle(malicious_email)
    malicious_ct = ct[:-16] + admin_last_block

    return malicious_ct

In [4]:
kv_parser('foo=bar&baz=qux&zap=zazzle')

{'foo': 'bar', 'baz': 'qux', 'zap': 'zazzle'}

In [50]:
import uuid
def profile_for(useremail):
    return f'email={useremail.replace("&", "").replace("=", "")}&uid={uuid.uuid4()}&role=user'
#     return f'email={useremail.replace("&", "").replace("=", "")}&uid=10&role=user'

In [51]:
profile_for("foo@bar.com")

'email=foo@bar.com&uid=ad09d424-ac2b-4130-a12a-ebb7fb5c4136&role=user'

In [52]:
profile_for("foo@bar.com&role=admin")

'email=foo@bar.comroleadmin&uid=733745d0-7ce1-48bb-a401-d8a4e5c786ff&role=user'

In [7]:
print(pad_pkcs7('Yellow Submarine'), unpad_pkcs7(pad_pkcs7('Yellow Submarine')))
print(pad_pkcs7('Yellow Submarine!!'), unpad_pkcs7(pad_pkcs7('Yellow Submarine!!')))

b'Yellow Submarine\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10' b'Yellow Submarine'
b'Yellow Submarine!!\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e' b'Yellow Submarine!!'


In [8]:
consistent_unknown_key = generate_random_bytes()
ct = encrypt_user_profile('foo@bar.com', consistent_unknown_key)
print('Ciphertext:', ct)
p = decrypt_user_profile(ct, consistent_unknown_key)
print('Decrypted profile:', p)

Ciphertext: b'q\x8c\x00\xb93\rP\x8c\xb8\xcf\x8c\xb9y$\xc1\x9c\x83e\xdfv\x97UX\xff\xaa\xac\x03F+\x87\x07\x16\x9c\x06M\xe9\xfe\x9a8e(\x1a6\xc4\x8fr(\x03'
Decrypted profile: {'email': 'foo@bar.com', 'uid': '10', 'role': 'user'}


In [47]:
def oracle(useremail):
    return encrypt_user_profile(useremail, consistent_unknown_key)
    
# probe for last block containing 'admin' and padding
probe_email = (AES.block_size - len('email=')) * chr(0) + pad_pkcs7('admin').decode()
admin_last_block = oracle(probe_email)[AES.block_size : 2*AES.block_size]

# probe block length
malicious_email = 'm@licio.us'
base_len = len(oracle(malicious_email))
while True:
    malicious_email = 'x'+malicious_email
    if len(oracle(malicious_email)) > base_len:
        break
malicious_email = 'x'*len('user') + malicious_email

# copy-and-paste admin block
ct = oracle(malicious_email)
malicious_ct = ct[:-16] + admin_last_block

print(malicious_email)
print(malicious_ct)
print(decrypt_user_profile(malicious_ct, consistent_unknown_key))

xxxxxxxxxxxxxxxxxxxm@licio.us
b'RX\x8a"GuO}\xc9lf*\t\xb4w\x95\xbe~\x8e\x88\xf7w\xc29\xfa\xc0\xac\x95J\ng\xc3\x84\xacr\xf3\'\xe6n\xd7\xd8zV\x8f\xe7\xc9\xd4<j\xda\x07\x0e+\xdb\xfff\x83B\x9a?v0\xb5\xaa'
{'email': 'xxxxxxxxxxxxxxxxxxxm@licio.us', 'uid': '10', 'role': 'admin'}


In [53]:
decrypt_user_profile(construct_admin_ciphertext(), consistent_unknown_key)

{'email': 'xxxxxxxxxxxxxxxxxm@licio.us',
 'uid': '63da1f76-1c94-4cb6-8c54-e8d715b92b35',
 'role': 'admin'}